# Starter Code

In [142]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [143]:
df_2018 = pd.read_csv('../data/2018.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2016 = pd.read_csv('../data/2016.csv')

def get_classifier_convert(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0),
        (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['CONVERTED', 'FAILED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def get_classifier_attempt(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0) | (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['ATTEMPTED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def ready_data_convert(df):
    df = df.dropna()
    y = get_classifier_convert(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

def ready_data_attempt(df):
    df = df.dropna()
    y = get_classifier_attempt(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

# Tree

## Convert 4th down

In [144]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(X_train, y_train)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

SyntaxError: unexpected EOF while parsing (3549174996.py, line 7)

## Attempt 4th down

In [ ]:
x, y = ready_data_attempt(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")
print(clf.feature_names_in_)
print(clf.feature_importances_)

Test Accuracy: 0.8849935870029927
Train Accuracy: 0.8790891597177678
['yardline_100' 'game_seconds_remaining' 'ydstogo' 'score_differential']
[0.78087865 0.06367388 0.07767985 0.07776762]
